In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm 

In [2]:
df = pd.read_csv('df_english.csv')

In [3]:
df.head(1)

,Unnamed: 0,Id,Price,Address,Nearest_Subway,Publication_date,Num_of_rooms,Total_area,Living_space,Floor,...,Cargo_lift,New_building_name,Building,Official_developer,Participation_type,Due_date,Link,Coordinates,WMO,Area
0,0,105724302,9800000,"Санкт-Петербург, Пушкинский р-н, пос. Шушары, ...",NaN,17.10.2022 в 10:16,3,73 м²,NaN,1 из 5,...,NaN,NaN,NaN,NaN,NaN,NaN,https://avito.ru//sankt-peterburg/kvartiry/3-k...,"59.737926, 30.461476",Шушары,Пушкинский


In [4]:
# Избавимся от признаков, которые не несут информативности для нас.
df = df.drop(['Unnamed: 0', 'Id', 'Official_developer', 
              'New_building_name', 'Building', 'Link', 'Floors_in_house'], axis=1)

# Избавляем от признаков, у которых пропусков более 65%
df = df.dropna(thresh=int(len(df) * .65), axis=1)

# Избавляемся от дубликатов
df = df.drop_duplicates(['Address', 'Total_area', 'Floor'])

После того как мы удалили все ненужные признаки, у нас остались следующие фичи:

In [5]:
df.columns

Index(['Price', 'Address', 'Nearest_Subway', 'Publication_date',
       'Num_of_rooms', 'Total_area', 'Living_space', 'Floor', 'Balcony',
       'Bathroom', 'Windows', 'Sale_method', 'House_type', 'Coordinates',
       'WMO', 'Area'],
      dtype='object')

С ними мы и будем работать.

# Feature_engineering

### Обработка площади

In [6]:
# убираем м.кв. из "Total_area" и "Living_space"
def convert_area(value):
    correct_list = [str(i) for i in range(10)] + ['.']
    result = ''
    if pd.isna(value):
        return value
    for literal in str(value):
        if literal in correct_list:
            result += literal
    return result

for col in ["Total_area", "Living_space"]:
    df.loc[:, col] = df[col].apply(convert_area).astype(np.float32)

### Обработка координат

Ориентироваться на адрес не очень удобно, поэтому этот признак преобразуем в кооридинаты

In [7]:
df[['lat_object', 'lon_object']] = df['Coordinates'].str.split(', ',expand=True).astype('float')

df[['lat_object', 'lon_object']].describe()

,lat_object,lon_object
count,19565.000000,19565.000000
mean,59.675410,30.485540
std,2.044360,3.272827
min,38.552623,-4.719892
25%,59.852538,30.251297
50%,59.931927,30.326360
75%,60.000654,30.406679
max,60.262019,137.986851


Видно, что в данных есть выбросы, не характерные для координат СПб. Ограничив последние диапазоном по широте от 58,7 до 61 град и по долготе от 28,5 до 33,5, введем фильтр.

In [8]:
df[(df['lat_object'] > 61) | (df['lat_object'] < 58.7) | 
   (df['lon_object'] > 33.5) | (df['lon_object'] < 28.5)]['Price'].count()

348

In [9]:
df = df.drop(df[(df['lat_object'] > 61) | (df['lat_object'] < 58.7) | 
   (df['lon_object'] > 33.5) | (df['lon_object'] < 28.5)].index)

In [10]:
#удаляем лишний столбец 
df = df.drop(["Coordinates"], axis=1)
# может еще понадобится удалить столбец Address

С КООРДИНАТАМИ НУЖНО ЕЩЕ ЗАМОРОЧИТЬСЯ И ИЗМЕРИТЬ РАССТОЯНИЕ ДО ЦЕНТРА!!!

### Обработка "Floor"

In [11]:
# разделим общее количество этажей в доме
# и квариру на конкретном этаже.
new_df = df['Floor'].str.split('из',expand=True)

#переименуем наши фичи
new_df.columns=['Flat_floor','Total_floor']

# объединим два датафрейма
df = pd.concat([df,new_df],axis=1)

# удалим ненужную фичу
df = df.drop(['Floor'], axis=1)

### Обработка Num_of_rooms

In [12]:
df.loc[df['Num_of_rooms'] == 'студия', ['Num_of_rooms']] = 0
df.loc[df['Num_of_rooms'] == '10 и больше', ['Num_of_rooms']] = 10
df = df.loc[df['Num_of_rooms'] != 'свободная планировка']

In [13]:
df['Num_of_rooms'].unique()

array(['3', '2', '1', 0, '4', '5', '7', '6', nan, 10, '8', '9'],
      dtype=object)

### Дополнительный Dataframe с координатами метро

Также нам понадобится еще один DataFrame с использованием координат местонахождения станций метро. Координаты в процессе работы также были нами спаршены.

In [14]:
path = 'C:/Users/Nekon/PetProject-price-real-estate-in-St.-Petersburg/parser/metro.csv'
subway_coordinates = pd.read_csv(path)
subway_coordinates.head()

,Станция,Координаты
0,Девяткино,"60.050182, 30.443045"
1,Гражданский проспект,"60.034969, 30.418224"
2,Академическая,"60.012806, 30.396044"
3,Политехническая,"60.008942, 30.370907"
4,Площадь Мужества,"59.999828, 30.366159"


In [15]:
#переименовываем название наших фичей
new_names = [
    "Station", "Coordinates"
 ]
mapper = {key:value for key, value in zip(subway_coordinates.columns.values, new_names)}

subway_coordinates.rename(columns=mapper, inplace=True)

In [16]:
#разделяем коррдинаты станций метро по широте и долготе
subway_coordinates[['lat_subway', 'lon_subway']] = subway_coordinates['Coordinates'].str.split(', ',expand=True).astype('float')

#переводим название станций метро в нижний регистр
subway_coordinates['Station'] = subway_coordinates['Station'].str.lower()

#удаляем уже ненужную фичу 'Station'
subway_coordinates = subway_coordinates.drop('Coordinates', axis=1)

subway_coordinates.head(1)

,Station,lat_subway,lon_subway
0,девяткино,60.050182,30.443045


Следующим шагом создадим функцию для расчета расстояния объекта до каждого метро

In [17]:
def destinator(latitiude_init, longitude_init, latitiude_fin, longitude_fin):
    '''
    Расчет расстояния в км
    '''
    return round(np.sqrt((latitiude_init - latitiude_fin)**2 + 
                        (longitude_init - longitude_fin)**2) * 111.13, 3)

def subway_distance_calculator(row, latitiude_fin, longitude_fin):
#    latitiude_init = row['Широта_Объекта']
#    longitude_init = row['Долгота_Объекта']
#
#    metro_distances = {}
#
#    for i in range(len(metro_coordinates)):
#        latitiude_fin = metro_coordinates['Широта'][i]
#        longitude_fin = metro_coordinates['Долгота'][i]
#
#        metro_distances[metro_coordinates['Станция'][i]] = [destinator(latitiude_init, longitude_init, latitiude_fin, longitude_fin)]
        

    return destinator(row['lat_object'], row['lon_object'], latitiude_fin, longitude_fin)

In [18]:
# Для корректной работы функции требуется сбросить индексы
df = df.reset_index(drop=True)

In [19]:
stations = ['расстояние_до_' + "_".join(x.split()) for x in subway_coordinates['Station']]
for i in tqdm(range(len(stations))):
    df[stations[i]] = df.apply(subway_distance_calculator, 
                               args=(subway_coordinates['lat_subway'][i], 
                                     subway_coordinates['lon_subway'][i]), 
                               axis=1
                              )

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:40<00:00,  1.78it/s]


In [20]:
df.head(1)

,Price,Address,Nearest_Subway,Publication_date,Num_of_rooms,Total_area,Living_space,Balcony,Bathroom,Windows,...,расстояние_до_адмиралтейская,расстояние_до_садовая,расстояние_до_звенигородская,расстояние_до_обводный_канал,расстояние_до_волковская,расстояние_до_бухарестская,расстояние_до_международная,расстояние_до_дунайская,расстояние_до_проспект_славы,расстояние_до_шушары
0,9800000,"Санкт-Петербург, Пушкинский р-н, пос. Шушары, ...",NaN,17.10.2022 в 10:16,3,73.0,NaN,балкон,раздельный,"во двор, на улицу, на солнечную сторону",...,27.35,26.37,25.042,23.334,21.026,19.195,17.306,12.66,15.105,9.662


### Обработка Nearest_Subway

Для начала разделим на 4 столца информацию о ближайших станциях метро

In [21]:
subway_stations = df['Nearest_Subway'].str.split(' мин.',expand=True)
subway_stations = subway_stations.fillna('no_info')
subway_stations.columns = ['subway_stations_1', 'subway_stations_2', 'subway_stations_3', 'subway_stations_4']
subway_stations

,subway_stations_1,subway_stations_2,subway_stations_3,subway_stations_4
0,no_info,no_info,no_info,no_info
1,Гостиный двор11–15,Невский проспект11–15,Маяковская11–15,
2,Комендантский проспект6–10,Старая деревняот 31,Пионерскаяот 31,
3,Приморскаяот 31,Василеостровскаяот 31,,no_info
4,Звёздная6–10,Купчино21–30,Московская21–30,
...,...,...,...,...
19185,Комендантский проспектот 31,,no_info,no_info
19186,Проспект Просвещенияот 31,Озеркиот 31,Гражданский проспектот 31,
19187,Пролетарская16–20,Обухово21–30,Ломоносовскаяот 31,
19188,Балтийская21–30,Московские ворота21–30,Фрунзенская21–30,


На первый взгляд видно, что в metro_station_4 есть только отсутствующие значения, проверим нашу гипотезу.

In [22]:
subway_stations['subway_stations_4'].unique()

array(['no_info', ''], dtype=object)

Предположения подтвердились. В таком случае, удаляем этот столбец.

In [23]:
subway_stations = subway_stations.drop('subway_stations_4', axis=1)

Отделим время до метро от названия станций.

In [24]:
def find_pattern(string: str, pattern: str):
    if re.search(pattern, string):
        return re.search(pattern, string).group(0)

def del_pattern(string: str, pattern: str):
    if re.search(pattern, string):
        return string.replace(pattern, "")
    else:
        return string

In [25]:
pattern_station_name = '^\D*'
pattern_duration = '\d\d$'
pattern_ot = 'от '
pattern_do = 'до '

In [26]:
subway_stations['subway_stations_1_duration'] = subway_stations['subway_stations_1'].apply(lambda x: find_pattern(x, pattern_duration))
subway_stations['subway_stations_2_duration'] = subway_stations['subway_stations_2'].apply(lambda x: find_pattern(x, pattern_duration))
subway_stations['subway_stations_3_duration'] = subway_stations['subway_stations_3'].apply(lambda x: find_pattern(x, pattern_duration))

subway_stations['subway_stations_1'] = subway_stations['subway_stations_1'].apply(lambda x: find_pattern(x, pattern_station_name))
subway_stations['subway_stations_2'] = subway_stations['subway_stations_2'].apply(lambda x: find_pattern(x, pattern_station_name))
subway_stations['subway_stations_3'] = subway_stations['subway_stations_3'].apply(lambda x: find_pattern(x, pattern_station_name))

subway_stations['subway_stations_1'] = subway_stations['subway_stations_1'].apply(lambda x: del_pattern(x, pattern_ot))
subway_stations['subway_stations_2'] = subway_stations['subway_stations_2'].apply(lambda x: del_pattern(x, pattern_ot))
subway_stations['subway_stations_3'] = subway_stations['subway_stations_3'].apply(lambda x: del_pattern(x, pattern_ot))

subway_stations['subway_stations_1'] = subway_stations['subway_stations_1'].apply(lambda x: del_pattern(x, pattern_do))
subway_stations['subway_stations_2'] = subway_stations['subway_stations_2'].apply(lambda x: del_pattern(x, pattern_do))
subway_stations['subway_stations_3'] = subway_stations['subway_stations_3'].apply(lambda x: del_pattern(x, pattern_do))

subway_stations = subway_stations.fillna('no_info')

In [27]:
subway_stations.head()

,subway_stations_1,subway_stations_2,subway_stations_3,subway_stations_1_duration,subway_stations_2_duration,subway_stations_3_duration
0,no_info,no_info,no_info,no_info,no_info,no_info
1,Гостиный двор,Невский проспект,Маяковская,15,15,15
2,Комендантский проспект,Старая деревня,Пионерская,10,31,31
3,Приморская,Василеостровская,,31,31,no_info
4,Звёздная,Купчино,Московская,10,30,30


In [29]:
subway_stations['subway_stations_1_duration'].unique()

array(['no_info', '15', '10', '31', '20', '30'], dtype=object)

In [31]:
# посмотрим какие названия станций у нас есть.
sorted(subway_stations['subway_stations_1'].unique().tolist())

['no_info',
 'Автово',
 'Адмиралтейская',
 'Академическая',
 'Балтийская',
 'Беговая',
 'Бухарестская',
 'Василеостровская',
 'Владимирская',
 'Волковская',
 'Выборгская',
 'Горьковская',
 'Гостиный двор',
 'Гражданский проспект',
 'Девяткино',
 'Достоевская',
 'Дунайская',
 'Елизаровская',
 'Звенигородская',
 'Звёздная',
 'Кировский завод',
 'Комендантский проспект',
 'Крестовский остров',
 'Купчино',
 'Ладожская',
 'Ленинский проспект',
 'Лесная',
 'Лиговский проспект',
 'Ломоносовская',
 'Маяковская',
 'Международная',
 'Московская',
 'Московские ворота',
 'Нарвская',
 'Невский проспект',
 'Новокрестовская',
 'Новочеркасская',
 'Обводный канал',
 'Обухово',
 'Озерки',
 'Парк Победы',
 'Парнас',
 'Петроградская',
 'Пионерская',
 'Площадь А. Невского I',
 'Площадь А. Невского II',
 'Площадь Восстания',
 'Площадь Ленина',
 'Площадь Мужества',
 'Политехническая',
 'Приморская',
 'Пролетарская',
 'Проспект Большевиков',
 'Проспект Ветеранов',
 'Проспект Просвещения',
 'Проспект Славы',
 

Переименуем названия в соответствии с их названиями в файле с координатами, а ткже названия райнов заменим на no_info

In [32]:
rename = {
    'Новокрестовская': 'Зенит',
    'Площадь А. Невского I': 'Площадь Александра Невского 1',
    'Площадь А. Невского II': 'Площадь Александра Невского 2',
    'Технологический ин-т I': 'Технологический институт',
    'Технологический ин-т II': 'Технологический институт 2',
    'р-н Калининский': 'no_info',
    'р-н Красногвардейский': 'no_info',
    'р-н Красносельский': 'no_info',
    'р-н Московский': 'no_info',
    'р-н Приморский': 'no_info',
    '': 'no_info',
}

In [33]:
subway_stations = subway_stations.replace(rename)

Теперь сохраним в датафрейм только столбцы с названиями станций и минимальным временем до метро (если время одинаковое для нескольких станций метро, будем указывать только одно ближайшее).

In [35]:
df[['Subway_station', 'Time_for_subway']] = subway_stations[['subway_stations_1', 'subway_stations_1_duration']]

In [37]:
df.head(5)

,Price,Address,Nearest_Subway,Publication_date,Num_of_rooms,Total_area,Living_space,Balcony,Bathroom,Windows,...,расстояние_до_звенигородская,расстояние_до_обводный_канал,расстояние_до_волковская,расстояние_до_бухарестская,расстояние_до_международная,расстояние_до_дунайская,расстояние_до_проспект_славы,расстояние_до_шушары,Subway_station,Time_for_subway
0,9800000,"Санкт-Петербург, Пушкинский р-н, пос. Шушары, ...",NaN,17.10.2022 в 10:16,3,73.0,NaN,балкон,раздельный,"во двор, на улицу, на солнечную сторону",...,25.042,23.334,21.026,19.195,17.306,12.660,15.105,9.662,no_info,no_info
1,24000000,"Санкт-Петербург, наб. реки Фонтанки, 28",Гостиный двор11–15 мин.Невский проспект11–15 м...,17.10.2022 в 01:22,3,85.0,NaN,NaN,совмещенный,"во двор, на солнечную сторону",...,2.547,2.767,5.043,6.779,8.641,13.331,10.838,16.558,Гостиный двор,15
2,12500000,"Санкт-Петербург, Камышовая ул., 3к1",Комендантский проспект6–10 мин.Старая деревняо...,22.10.2022 в 10:29,2,55.0,NaN,лоджия,раздельный,на улицу,...,12.626,14.534,16.742,18.602,20.489,25.331,22.808,28.624,Комендантский проспект,10
3,14200000,"Санкт-Петербург, ул. Беринга, 1",Приморскаяот 31 мин.Василеостровскаяот 31 мин.,21.10.2022 в 16:53,1,50.0,32.0,балкон,совмещенный,NaN,...,9.105,11.257,12.839,14.521,16.203,20.883,18.485,24.129,Приморская,31
4,5500000,"Санкт-Петербург, Пулковская ул., 8к2",Звёздная6–10 мин.Купчино21–30 мин.Московская21...,16.10.2022 в 03:52,0,26.0,14.0,балкон,совмещенный,"во двор, на солнечную сторону",...,9.518,8.531,6.473,5.414,4.676,6.504,5.234,9.174,Звёздная,10


### Вытаскиваем уникальные значения из некоторых категориальных фичей

In [ ]:
# #это метод OHE.
# def convert_lists(df: pd.DataFrame) -> pd.DataFrame:
#     cols = [
#         "Balcony", "Bathroom", "Windows", "Sale_method",
#         "House_type"
#            ]
#     new_cols = []
#     for col in cols:
#         series = df.loc[:, col]
#         data = []
#         unique_values = set()
#         new_cols_data = []

#         for row in series.values:
#             row_c = [value.strip() for value in (row.split(",") if isinstance(row, str) else [])]
#             for element in row_c:
#                 if not element in unique_values:
#                     unique_values.add(element)
#             data.append(row_c)
    
#         unique_values = list(unique_values)

#         for row in data:
#             new_cols_data.append([1 if value in row else 0 for value in unique_values])
    
#         col_names = [f"{col}_{i+1}" for i in range(len(unique_values))]
#         new_cols.append(pd.DataFrame(data=new_cols_data, columns=col_names, dtype=np.int8))
#         print("\n".join(map(str,zip(unique_values, col_names))))

#         df.drop(columns=[col], axis=1, inplace=True)

#     return pd.concat([df]+new_cols, axis=1)

# df = convert_lists(df)